In [1]:
 !pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.2 MB/s 
     |████████████████████████████████| 66 kB 4.9 MB/s 


In [2]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

# Definimos el paquete json para comunicarnos con unity
import json

In [3]:
cmap = matplotlib.cm.get_cmap('viridis', 8) # Puede ser cualquier otra
cmap = cmap(np.linspace(0, 1, 8))
cmap[0] = np.array([50/256, 50/256, 50/256, 1])   # Calle en gris
cmap[1] = np.array([256/256, 0/256, 0/256, 1])    # Semáforo en rojo
cmap[2] = np.array([256/256, 256/256, 0/256, 1])  # Semáforo en amarillo
cmap[3] = np.array([0/256, 256/256, 0/256, 1])    # Semáforo en verde
cmap[4] = np.array([0/256, 50/256, 100/256, 1])   # Carro azul
cmap[5] = np.array([230/256, 100/256, 20/256, 1]) # Carro naranja
cmap[6] = np.array([0/256, 0/256, 0/256, 1])  # Carro negro
cmap[7] = np.array([96/256, 170/256, 70/256, 1])  # No car zone


new_cmap = matplotlib.colors.ListedColormap(cmap)

contDown = 0
contRight = 0

In [4]:
#Carlos Gabriel Bazan Garcia A01284477
#Juan Eduardo Aran Flores A01284238
#Mariana Cortez de la Rosa A00830222
#Abiel Adrian Lozano Herrejon A01284213
#Fransisco Jose Joven Sanchez A00830564


class Terreno(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.live = 1

class Semaforo(Agent):
  """
  Representa a un agente del Semaforo
  """
  def __init__(self, unique_id, model, direccion=None):
    super().__init__(unique_id, model)
    self.color = "red"
    self.direccion = None

    if self.unique_id == 0:
      self.direccion = "right"
    elif self.unique_id == 1:
      self.direccion = "left"
    elif self.unique_id == 2:
      self.direccion = "up"
    else:
      self.direccion = "down"

  def step(self):
    pass
    
class Carro(Agent):
  DIRECTIONS = ['right', 'down', 'left', 'up']
  """
  Representa a un agente del Carro
  """
  def __init__(self, unique_id, model, direction=None):
    super().__init__(unique_id, model)
    self.pos = (0,0)
    self._direction = None
    self.next_pos = None
    self.color = random.choice(["color1","color2","color3"])

    self.direction = self.random.choice(self.DIRECTIONS) if not direction else direction

  @property
  def direction(self):
    return self._direction

  @direction.setter
  def direction(self, direction):
    self._direction = direction
    if self._direction == 'up':
      self.dx, self.dy = 0, -1
    elif self._direction == 'down':
      self.dx, self.dy = 0, 1
    if self._direction == 'right':
      self.dx, self.dy = -1, 0
    elif self._direction == 'left':
      self.dx, self.dy = 1, 0

  def step(self):
    next_pos = (self.pos[0] + self.dx, self.pos[1] + self.dy)

    if self.model.grid.out_of_bounds(next_pos):
      self.next_pos = self.model.grid.torus_adj(next_pos)
    else:
      self.next_pos = next_pos

  def advance(self):
      x = True
      neighbors = self.model.grid.get_neighbors(self.pos, moore=False, include_center=False)

      for n in neighbors:
        if self.direction == "up" or self.direction == "down":
          if isinstance(n, Semaforo):
            if n.unique_id == 0 or n.unique_id == 1: 
              if n.color == "red" or n.color == "yellow":
                x = False

        elif self.direction == "right" or self.direction == "left":
          if isinstance(n, Semaforo):
            if n.unique_id == 2 or n.unique_id == 3: 
              if n.color == "red" or n.color == "yellow":
                x = False

      if self.model.grid.is_cell_empty(self.next_pos) and x:
        self.model.grid.move_agent(self, self.next_pos)
        self.pos = self.next_pos
      

class Interseccion(Model):
  """
  Define el modelo del juego de la vida.
  """
  def __init__(self, width, height):
    self.temp = 0
    self.inicialTime = 5
    self.num_agents = width * height
    
    self.grid = SingleGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)    
    
    cont = 0
    self.sema = list()
    for x, y in [((width//2 - 2), (height//2 + 1)), ((width//2 + 1), (height//2 - 2)), ((width//2 + 1), (height//2 + 1)), ((width//2 - 2), (height//2 - 2))]:
      s = Semaforo(cont, self)
      if cont == 0:
        s.color = "green"
      #self.schedule.add(s)
      self.grid.place_agent(s, (x, y))
      cont += 1
      self.sema.append(s)


    #donde = ('right', 'down')                  Se iniciliza el primer carro
    seleccion = self.random.choice(Carro.DIRECTIONS)
    carro = Carro(cont, self, direction=seleccion)
    self.schedule.add(carro)
    cont += 1

    if seleccion == "right":
        pos = (0, height//2)  
    elif seleccion == "down":
        pos = (width//2, 0)
    elif seleccion == "left":
        pos = (width-1, height//2-1)
    elif seleccion == "up":
        pos = (width//2-1, height-1)
    
    self.grid.place_agent(carro, pos)

    ######
    for content, x, y in self.grid.coord_iter():
      if (not content) and (x not in [width//2-1, width//2]) and (y not in [height//2-1, height//2]):
        t = Terreno(f"t{cont}", self)
        self.grid.place_agent(t, (x,y))
        self.schedule.add(t)
        cont += 1
    ######

    # Aquí definimos el colector de datos para obtener el grid completo.
    self.datacollector = DataCollector(
        model_reporters={"Grid": self.get_grid,
                       "JSON_step": self.read_agents}
    )


  def place_car(self, cont, size, contCar):   
    global contDown
    global contRight
    seleccion = self.random.choice(Carro.DIRECTIONS)
    carro = Carro(cont, self, direction=seleccion)
    self.schedule.add(carro)

    if seleccion == "right":
        pos = (0, size//2)  
    elif seleccion == "down":
        pos = (size//2, 0)
    elif seleccion == "left":
        pos = (size-1, size//2-1)
    elif seleccion == "up":
        pos = (size//2-1, size-1)

    if seleccion == "right": #and contRight < contCar:
        if self.grid.is_cell_empty(pos):
            self.grid.place_agent(carro, pos)
    if seleccion == "down": #and #contDown < contCar:
        if self.grid.is_cell_empty(pos):
            self.grid.place_agent(carro, pos)
    if seleccion == "left": #and #contRight < contCar:
        if self.grid.is_cell_empty(pos):
            self.grid.place_agent(carro, pos)
    if seleccion == "up": #and #contDown < contCar:
        if self.grid.is_cell_empty(pos):
            self.grid.place_agent(carro, pos)
  
  def step(self, cont, size, contCar):
    """
    En cada paso el colector toma la información que se definió y almacena el grid para luego
    graficarlo.
    """
    self.datacollector.collect(self)
    self.schedule.step()
    if cont%3 == 0 and cont < 260:
        self.place_car(cont, size, contCar)

    if self.temp < self.inicialTime:
      self.temp += 1
    else:
      for i, sem in enumerate(self.sema):
        if sem.color == "green":
          self.sema[i].color = "red"
          self.temp = 0
          if i < 3:
            nextSem = i+1
          else:
            nextSem = 0
          self.sema[nextSem].color = "green"
          break

  def get_grid(self):
    """
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    :param model: El modelo del cual obtener el grid.
    :return: Matriz con la información del grid del agente.
    """

    # Generamos la grid para contener los valores
    grid = np.zeros((self.grid.width, self.grid.height))

    # Asignamos una celda a cada uno de los elementos de la grilla
    
    for cell in self.grid.coord_iter():
      cell_content, x, y = cell
      if isinstance(cell_content, Carro):
        if cell_content.color == "color1":
            grid[x][y] = 4
        elif cell_content.color == "color2":
            grid[x][y] = 5
        elif cell_content.color == "color3":
            grid[x][y] = 6
      elif isinstance(cell_content, Semaforo):
        if cell_content.color == "yellow":
            grid[x][y] = 2 # amarillo 
        elif cell_content.color == "green":
            grid[x][y] = 3 # verde
        elif cell_content.color == "red":
            grid[x][y] = 1 # rojo
      elif isinstance(cell_content, Terreno): 
        grid[x][y] = 7
      else: 
        grid[x][y] = 0    

    return grid

  def read_agents(self):
    agents_list = []
    for cell in self.grid.coord_iter():
        agent, x, y = cell

        if isinstance(agent, Carro):
            agent_dict = {'id': agent.unique_id,'kind': 'carro','positionX': agent.pos[0],'positionY': 0,'positionZ': agent.pos[1],'color': agent.color}
        elif isinstance(agent, Semaforo):
            agent_dict = {'id': agent.unique_id,'kind': 'semaforo','positionX': agent.pos[0],'positionY': 0,'positionZ': agent.pos[1],'color': agent.color}
        else:
            agent_dict = {}


        if len(agent_dict) != 0:
            agents_list.append(agent_dict)

    return agents_list

In [5]:
# Definimos el tamaño del Grid
GRID_SIZE = 12

# Definimos la cantidad de carros a generar
CAR_CONT = 2

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 100

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = Interseccion(GRID_SIZE, GRID_SIZE)

contCarros = 200

archivo = open("test.json", "w")

for i in range(NUM_GENERATIONS):
    model.step(contCarros, GRID_SIZE, CAR_CONT)
    contCarros += 1

    jsn = model.read_agents()

    for json_object in jsn:
        archivo.write(json.dumps(json_object, indent=4))

archivo.close()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.092981


In [6]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [7]:
all_grid["JSON_step"].to_json("test.json")

In [8]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [9]:
anim